In [11]:
import time
from multiprocessing.managers import SharedMemoryManager
import click
import cv2
import numpy as np
import torch
import dill
import hydra
import pathlib
import skvideo.io
from omegaconf import OmegaConf
import scipy.spatial.transform as st
from diffusion_policy.common.precise_sleep import precise_wait
from diffusion_policy.real_world.real_inference_util import (
    get_real_obs_resolution,
    get_real_obs_dict,
)
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.base_image_policy import BaseImagePolicy
from diffusion_policy.common.cv2_util import get_image_transform

from diffusion_policy.dataset.h2r_dataset import H2RDataset

OmegaConf.register_new_resolver("eval", eval, replace=True)

In [4]:
ckpt_path = "data/outputs/2025.04.12/08.22.59_train_diffusion_unet_hybrid_square_image/checkpoints/latest.ckpt"
payload = torch.load(open(ckpt_path, "rb"), pickle_module=dill)
cfg = payload["cfg"]
cls = hydra.utils.get_class(cfg._target_)

In [24]:
task_cfg = cfg.task.dataset

new_cfg = {**task_cfg}
new_cfg["dataset_path_list"] = [
    "/home/ubuntu/project/Moore-AnimateAnyone/data/writing/writing_cvpr/episode_0.hdf5"
]
new_cfg["is_task"] = False
new_cfg.pop("_target_")
print(new_cfg)

a = dict()
dataset = H2RDataset(**new_cfg)

{'shape_meta': {'obs': {'human': {'shape': [3, 240, 426], 'type': 'rgb'}, 'robot0_eef_pos': {'shape': [3]}, 'robot0_eef_rot_pos': {'shape': [3]}, 'robot0_gripper_qpos': {'shape': [1]}}, 'action': {'shape': [10]}}, 'dataset_path_list': ['/home/ubuntu/project/Moore-AnimateAnyone/data/writing/writing_cvpr/episode_0.hdf5'], 'base_dataset_path': '/data1/dataset', 'horizon': 4, 'pad_before': 3, 'pad_after': 3, 'n_obs_steps': 4, 'abs_action': True, 'rotation_rep': 'rotation_6d', 'use_legacy_normalizer': False, 'use_cache': True, 'seed': 42, 'val_ratio': 0.02, 'is_task': False}
Acquiring lock on cache.
Loading cached ReplayBuffer from Disk.
action
human
robot
robot0_eef_pos
robot0_eef_rot_pos
robot0_gripper_qpos
Loaded!


In [29]:
dataset[8]['action']

tensor([[160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000]])

In [30]:
dataset[7]["action"]

tensor([[160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000]])

In [31]:
dataset[6]["action"]

tensor([[160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [160.6198, 344.0589, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000],
        [158.3165, 344.8860, 123.0000,  -0.4481,   0.0000,   0.8940,  -0.7992,
          -0.4481,  -0.4006,   0.0000]])